# Debugging Faster R-CNN Networks

### Imports

In [1]:
import sys
sys.path.append('../')

import numpy as np
import tensorflow as tf

### Check Cythonized modules working

#### bbox_overlaps

In [2]:
from Lib.bbox_overlaps import bbox_overlaps

In [3]:
boxes = np.array([[1,3,2,4],
                  [2,5,1,8],
                  [3,4,9,9],
                  [1,9,7,11]],dtype=np.float)

In [4]:
query_boxes = np.array([[1,3,2,4],
                  [5,3,7,4],
                  [1,8,10,12],
                  [0,2,3,5],
                  [0,3,2,9],
                  [2,1,3,4]],dtype=np.float)

In [5]:
bbox_overlaps(boxes,query_boxes)

array([[ 1.        ,  0.        ,  0.        ,  0.25      ,  0.19047619,
         0.2       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.06666667,  0.17948718,  0.03571429,  0.        ,
         0.02040816],
       [ 0.        ,  0.        ,  0.42      ,  0.        ,  0.05      ,
         0.        ]])

#### nms

In [6]:
from Lib.nms_wrapper import nms

In [7]:
proposals = 10*np.array([[2,1,12,21,0],
                     [10,5,14,18,.1],
                     [5,3,11,14,.2],
                     [1,4,20,20,.3]],dtype=np.float32)
copies = np.random.choice(4,256,p=[0.1,0.2,0.3,0.4])
proposals = proposals[copies,:]
perturbations = np.random.randint(8,size=(256,4))
perturbations = np.concatenate((perturbations,np.zeros([256,1])),1)
proposals = np.array(proposals + perturbations,dtype=np.float32)

In [8]:
keep = nms(proposals,0.7)
proposals[keep,:]

array([[  13.,   44.,  205.,  205.,    3.],
       [  50.,   37.,  112.,  144.,    2.],
       [ 102.,   55.,  147.,  181.,    1.],
       [  24.,   10.,  122.,  216.,    0.]], dtype=float32)

### Test fast_rcnn

In [9]:
from Networks.convnet import convnet
from Networks.faster_rcnn_networks import rpn, roi_proposal, fast_rcnn

In [10]:
# Global Dictionary of Flags
flags = {
    'data_directory': '../Data/MNIST/',
    'save_directory': '../Logs/summaries/',
    'model_directory': 'resnet101/',
    'restore': False,
    'restore_file': 'start.ckpt',
    'datasets': 'MNIST',
    'image_dim': 28,
    'hidden_size': 10,
    'num_classes': 10,
    'batch_size': 1,
    'display_step': 200,
    'weight_decay': 1e-7,
    'lr_decay': 0.999,
    'num_epochs': 10,
    'lr_iters': [(5e-3, 5000), (5e-3, 7500), (5e-4, 10000), (5e-5, 10000)],
    'anchor_scales': [8,16,32]
}

#### Set up inputs

In [11]:
x = tf.placeholder(tf.float32, [None, 128, 128, 3], name='x')
gt = tf.placeholder(tf.int64, [None, 5], name='gt')
im_dims = tf.placeholder(tf.int64, [None, 2], name='im_dims')

#### Convolutional Feature Extractor

In [12]:
cnn = convnet(x, [5, 3, 3, 3, 3], [64, 96, 128, 172, 256, 512], strides=[2, 2, 2, 2, 2, 2])
featureMaps = cnn.get_output()
_feat_stride = cnn.get_feat_stride()

conv_1 output: (?, 64, 64, 64)
conv_2 output: (?, 32, 32, 96)
conv_3 output: (?, 16, 16, 128)
conv_4 output: (?, 8, 8, 172)
conv_5 output: (?, 4, 4, 256)


#### RPN

In [13]:
rpn_net = rpn(featureMaps,gt,im_dims,_feat_stride,flags)

conv_1 output: (?, 4, 4, 512)
conv_1 output: (?, 4, 4, 18)
conv_1 output: (?, 4, 4, 36)


In [14]:
rpn_cls_score = rpn_net.get_rpn_cls_score()
rpn_cls_score.get_shape()

TensorShape([Dimension(None), Dimension(4), Dimension(4), Dimension(18)])

In [15]:
rpn_labels = rpn_net.get_rpn_labels()
rpn_labels.get_shape()

TensorShape(None)

In [16]:
rpn_bbox_pred = rpn_net.get_rpn_bbox_pred()
rpn_bbox_pred.get_shape()

TensorShape([Dimension(None), Dimension(4), Dimension(4), Dimension(36)])

In [17]:
rpn_bbox_targets = rpn_net.get_rpn_bbox_targets()
rpn_bbox_targets.get_shape()

TensorShape(None)

#### ROI Proposal

In [18]:
rpn_cls_score = rpn_net.get_rpn_cls_score()
rpn_bbox_pred = rpn_net.get_rpn_bbox_pred()
roi_proposal_net = roi_proposal(rpn_cls_score, rpn_bbox_pred, gt, im_dims, flags)

#### Fast RCNN

In [ ]:
fast_rcnn_net = fast_rcnn(featureMaps, roi_proposal_net)

#### Feed dummy values through net

In [19]:
config = tf.ConfigProto(log_device_placement=False)
config.gpu_options.per_process_gpu_memory_fraction=0.2
sess = tf.InteractiveSession(config=config)